In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
#from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
#from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)

PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 10
NUM_EVAL = 10
CONTINUAL = True

def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
eval_env = MyEnv({})
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env

config = ppo.DEFAULT_CONFIG.copy()
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 3000*5,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          #"clip_actions":True,"normalize_actions":True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 300, "num_sgd_iter":20,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":1200*5*5}, #PPO デフォルト "random_timesteps":0
          #"model":{"fcnet_activation": "relu","fcnet_hiddens": [256, 256, 256],"post_fcnet_activation": "linear",
          #         "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          #"model":{"vf_share_layers": True,"use_lstm": True},
          #"model": {"custom_model": "my_model"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
#res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"

conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    #model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000121/checkpoint-121'
    
    trainer.restore(checkpoint_path=model_path)

# Instanciate the evaluation env
eval_env = MyEnv({})
record_mode = 0

C:\Users\DMU\miniconda3\envs\AI2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DMU\miniconda3\envs\AI2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DMU\miniconda3\envs\AI2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DMU\miniconda3\envs\AI2\lib\site-packages\tensorflow\python\framework\dty

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2022-02-21 18:52:01,425	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-02-21 18:52:01,776	INFO trainable.py:383 -- Restored on 192.168.56.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_001516\checkpoint-1516
2022-02-21 18:52:01,776	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 1516, '_timesteps_total': None, '_time_total': 1842482.89387393, '_episodes_total': 42933}


In [2]:
for i in range(NUM_EVAL):
    # print(f'\nEvaluation {i}:')
    obs = eval_env.reset()
    done = False
    eval_env.eval = True
    step_num = 0
    fig = plt.figure(1,figsize=(8.0, 6.0))
    ESC = 0x1B          # ESCキーの仮想キーコード
    trajectory_length = 100
    env_blue_pos = [0]
    env_red_pos = [0]
    env_mrm_pos = [0]
    if record_mode == 0:
        file_name = "test_num" +str(i)
        video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

    while True:
        action_dict = {}
        #for j in range(eval_env.blue_num):
            #if not eval_env.blue[j].hitpoint == 0:
                #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                                                                   #clip_actions=True,explore=False)
                #print(action_dict['blue_' + str(j)]["vector_psi_x"],action_dict['blue_' + str(j)]["vector_gam_x"],
                      #action_dict['blue_' + str(j)]["velocity"])
        #obs, rewards, dones, infos = eval_env.step(action_dict)
        action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0',explore=False)
        action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1',explore=False)
            #action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0')
            #action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1')
        obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0, 'blue_1': action_dict1})
        #for j in range(eval_env.blue_num): 

            #print('blue_' + str(j),eval_env.blue[j].V, np.rad2deg(eval_env.blue[j].psi),  np.rad2deg(eval_env.blue[j].gam),
            #     eval_env.blue[j].V_ref,np.rad2deg(eval_env.blue[j].psi_ref),  np.rad2deg(eval_env.blue[j].gam_ref))
        env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
        env_blue_pos.append(env_blue_pos_temp)
        env_red_pos.append(env_red_pos_temp)
        env_mrm_pos.append(env_mrm_pos_temp)
        if step_num == 0:
            del env_blue_pos[0]
            del env_red_pos[0]
            del env_mrm_pos[0]

        hist_blue_pos = np.vstack(env_blue_pos)
        hist_red_pos = np.vstack(env_red_pos)
        hist_mrm_pos = np.vstack(env_mrm_pos)
        plt.clf()
        render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
        render_env.rend_3d(eval_env,hist_red_pos,"r",1)
        render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
        plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
        plt.pause(.01)
        step_num = step_num + 1

        done = dones["__all__"]
        if record_mode == 0:
            img = np.array(fig.canvas.renderer.buffer_rgba())
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
            # cv2.imshow('test', img)
            # cv2.waitKey(1)
            # cv2.destroyAllWindows()
            video.write(img.astype('uint8'))
        elif record_mode == 1:
            env.render()
        elif record_mode == 2:
            True #将来のため処理予約

        #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)

        #env_blue_pos.append(env_blue_pos_temp)
        #env_red_pos.append(env_red_pos_temp)
        #env_mrm_pos.append(env_mrm_pos_temp)
        #step_num = step_num + 1
        # エピソードの終了処理
        if dones['__all__']:
            # print(f'all done at {env.steps}')
            if record_mode == 0:
                video.release()
            break

    #del env_blue_pos[0]
    #del env_red_pos[0]
    #del env_mrm_pos[0]

    #hist_blue_pos = np.vstack(env_blue_pos)
    #hist_red_pos = np.vstack(env_red_pos)
    #hist_mrm_pos = np.vstack(env_mrm_pos)

    #f = open(results_file,'wb')
    #f = open("log.pkl","wb")
    #pickle.dump(emv_blue_pos,f)
    #pickle.dump(emv_blue_pos,f)
    #pickle.dump(emv_red_pos,f)
    #pickle.dump(emv_mrm_pos,f)
    #f.close()

    if record_mode == 0:
        video.release()

ray.shutdown()

-------------------------- Scene: 1 --------------------------
Blue_0Too Close Finish
TIME LIMIT LOSE
blue_0 False False 673 -0.19 44.250000000000156
blue_1 False False 673 0.0 24.3612
-------------------------- Scene: 1 --------------------------
Blue_0Too Close Finish
TIME LIMIT LOSE
blue_0 False False 671 -0.19 43.6900000000001
blue_1 False False 671 0.0 24.282000000000007
-------------------------- Scene: 0 --------------------------
Blue_0Too Close Finish
TIME LIMIT LOSE
blue_0 False False 420 -0.11 17.529999999999973
blue_1 False False 420 0.0804 7.368000000000002
-------------------------- Scene: 1 --------------------------
Blue_0Too Close Finish
TIME LIMIT LOSE
blue_0 False False 671 -0.19 45.090000000000046
blue_1 False False 671 0.0 25.288600000000006
-------------------------- Scene: 1 --------------------------
Blue_0Too Close Finish
TIME LIMIT LOSE
blue_0 False False 672 -0.19 43.830000000000055
blue_1 False False 672 0.0 22.712999999999983
-------------------------- Scen

C:\Users\DMU\AppData\Roaming\Python\Python37\site-packages\ipykernel\eventloops.py:258: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()
